In [1]:
from fetch_crypto_data import fetch_crypto_data
from scrape_bitcoin_news import scrape_bitcoin_news


Bitcoin Data (USD):
             1. open   2. high    3. low  4. close     5. volume
2023-07-29  29316.05  29411.09  29253.44  29355.71   2893.052988
2023-07-30  29356.65  29450.00  29033.00  29281.39   4387.572566
2023-07-31  29280.41  29526.99  29102.65  29230.67   8793.063285
2023-08-01  29230.61  29726.73  28477.00  29697.27  17752.839341
2023-08-02  29699.25  30033.00  28903.79  29164.22  17967.347996
          Date                                              Title
0   2024-05-23  ETFs shift from ‘core’ to ‘satellite’ in portf...
1   2024-05-22           First UK crypto ETPs to launch on May 28
2   2024-05-22         UK’s first crypto ETPs to launch on May 28
3   2024-05-22  Transcript: PwC’s Evergrande audit comes back ...
4   2024-05-21     Ether jumps 16% on US ETF approval speculation
5   2024-05-21  Markets update: Hong Kong stocks fall as prope...
6   2024-05-20  Edited highlights from the evisceration of Cra...
7   2024-05-20  Head of crypto asset manager Grayscale steps d

In [2]:
import psycopg2
from psycopg2 import sql


In [3]:
from dotenv import load_dotenv
import os


load_dotenv()


True

In [4]:
# Database connection parameters
DATABASE_URL = os.getenv('DATABASE_URL')
# Connect to the database
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()


# Create tables if they don't exist
cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_data(
    date DATE PRIMARY KEY,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_news2 (
    id SERIAL PRIMARY KEY,
    Date DATE,
    Title TEXT,
    UNIQUE (Date, Title)
)
''')

conn.commit()

In [5]:
# Function to insert data into the bitcoin_data table
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO bitcoin_data (date, open, high, low, close, volume)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (date) DO NOTHING
            ''', (index, row['1. open'], row['2. high'], row['3. low'], row['4. close'], row['5. volume']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f"Error inserting bitcoin data: {e}")
        conn.rollback()

In [6]:
# Function to insert data into the bitcoin_news table
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO bitcoin_news (id, title, date)
                VALUES (%s, %s, %s)
                ON CONFLICT (id) DO NOTHING
            ''', (index, row['Title'], row['Date']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f"Error inserting bitcoin news: {e}")
        conn.rollback()

In [7]:


bitcoin_news_df = scrape_bitcoin_news()
display(bitcoin_news_df)

,Date,Title
0,2024-05-23,ETFs shift from ‘core’ to ‘satellite’ in portf...
1,2024-05-21,Ether jumps 16% on US ETF approval speculation
2,2024-05-21,Markets update: Hong Kong stocks fall as prope...
3,2024-05-14,BlackRock closes in on crown of world’s larges...
4,2024-05-14,FirstFT: US piles tariffs on to Chinese EVs an...
...,...,...
100,2024-02-20,Lavish Miami ETF party highlights how far indu...
101,2024-02-19,Defendant had ‘no idea’ bitcoin funds were pro...
102,2024-02-19,Has Gensler’s SEC pushed Wall Street too far?
103,2024-02-16,The crypto lobby’s newest target


In [8]:
# Main function to orchestrate the data fetching and insertion
def main():
    # Fetch Bitcoin data
    ##bitcoin_data = fetch_crypto_data('BTC', 'USD')

    # Scrape Bitcoin news data
    bitcoin_news_df = scrape_bitcoin_news()

    # Connect to the PostgreSQL database
    conn = psycopg2.connect(DATABASE_URL)

    if conn:
        # Create tables if they do not exist
        #create_tables(conn)

        # Insert data into the database
        ##insert_bitcoin_data(conn, bitcoin_data)
        insert_bitcoin_news(conn, bitcoin_news_df)

        # Close the database connection
        conn.close()
    else:
        print("Failed to connect to the database.")

if __name__ == '__main__':
    main()